# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [7]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet','stopwords','averaged_perceptron_tagger'])

class style:
    BOLD = '\033[1m'
    END = '\033[0m'
    
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import string
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline,FeatureUnion

from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,make_scorer,classification_report,fbeta_score

from sklearn.base import BaseEstimator, TransformerMixin
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X =  df.message
y = (df[df.columns[4:]])
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df[df.columns[4:]].head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [26]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [4]:
len(df.columns[4:]),df.columns[4:]

(36, Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
        'medical_products', 'search_and_rescue', 'security', 'military',
        'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
        'missing_people', 'refugees', 'death', 'other_aid',
        'infrastructure_related', 'transport', 'buildings', 'electricity',
        'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
        'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
        'other_weather', 'direct_report'],
       dtype='object'))

In [5]:
X[:5]

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    '''
    Normalize,tokenize,apply stemming and lemmatization to text data.
    
    Args: text(str) containing message that needs to be tokenized.
    
    Returns : processed word tokens-normalized,tokenized,lemmatized and stemmed
    '''
    #message = text.translate(str.maketrans('','',string.punctuation)).lower() #normalize
    message=re.sub(r"[^a-zA-Z0-9]"," ",text)
    words = word_tokenize(message) # tokenize
    words = [w for w in words if w not in stopwords.words('english')] #apply stopwords
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words] #lemmatize
    stemmed = [PorterStemmer().stem(w).lower().strip() for w in lemmed] # stem

    return stemmed

In [10]:
pd.set_option('display.max_colwidth', -1)
msg=df[df.id==2]['message']
msg[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [5]:
tokenize(msg[0])

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=1))),
])
pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...1,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=1))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
#Create train and test data
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=1)

# train classifier
pipeline.fit(X_train,y_train)

# evaluate all steps on train data
predicted_tr = pipeline.predict(X_train)

**Apply the classification_report to get the precision,recall and f1-score on the train data**

In [7]:
predicted_tr_df=pd.DataFrame(predicted_tr,columns=df.columns[4:])
for col in df.columns[4:]:
    print(style.BOLD + 'Feature : {}\n'.format(col) + style.END,classification_report(y_train[col],predicted_tr_df[col]) + '\n')

Feature : related
              precision    recall  f1-score   support

          0       0.99      0.97      0.98      4572
          1       0.99      1.00      0.99     14955
          2       0.99      0.90      0.94       135

avg / total       0.99      0.99      0.99     19662


Feature : request
              precision    recall  f1-score   support

          0       0.98      1.00      0.99     16327
          1       1.00      0.92      0.96      3335

avg / total       0.99      0.99      0.99     19662


Feature : offer
              precision    recall  f1-score   support

          0       1.00      1.00      1.00     19570
          1       1.00      0.72      0.84        92

avg / total       1.00      1.00      1.00     19662


Feature : aid_related
              precision    recall  f1-score   support

          0       0.98      1.00      0.99     11541
          1       0.99      0.97      0.98      8121

avg / total       0.98      0.98      0.98     19662


Featu

**Fbeta  and Accuracy score**

In [8]:
print(style.BOLD + 'F-beta score for the features \n' + style.END)
f_score_lst=[]
for col in df.columns[4:]:
    f_score_lst.append(fbeta_score(y_train[col],predicted_tr_df[col],average='weighted',beta=1))
    print(style.BOLD + col + style.END,':', fbeta_score(y_train[col],predicted_tr_df[col],average='weighted',beta=1))
    
print(style.BOLD +'\nMean fbeta score : {}'.format(np.mean(f_score_lst)) +style.END)    

F-beta score for the features 

related : 0.989071088573
request : 0.986471235977
offer : 0.998569290481
aid_related : 0.984761486581
medical_help : 0.987602456083
medical_products : 0.99134508534
search_and_rescue : 0.99365887793
security : 0.99464640463
military : 0.995704910381
child_alone : 1.0
water : 0.994818426623
food : 0.993302659556
shelter : 0.991633440883
clothing : 0.997899276324
money : 0.994784218347
missing_people : 0.996944051834
refugees : 0.992526673227
death : 0.994239733409
other_aid : 0.977454140443
infrastructure_related : 0.986529759541
transport : 0.990531132913
buildings : 0.992073873117
electricity : 0.996157924523
tools : 0.997604053078
hospitals : 0.996706000349
shops : 0.998747019463
aid_centers : 0.99696404084
other_infrastructure : 0.990105574842
weather_related : 0.987214674702
floods : 0.991885874393
storm : 0.994284832628
fire : 0.997342171364
earthquake : 0.995900293353
cold : 0.997159455702
other_weather : 0.989000095147
direct_report : 0.9813039143

In [9]:
acc_score_lst=[]
for col in df.columns[4:]:
    acc_score_lst.append((predicted_tr_df[col].values==y_train[col].values).mean())
    
print(style.BOLD +'Mean acc score : {}'.format(np.mean(acc_score_lst)) +style.END)  

Mean acc score : 0.9928782535968987


In [13]:
#ALternatively
# acc_score_2=[]
# for col in df.columns[4:]:
#     acc_score_2.append((accuracy_score(y_train[col].values,predicted_tr_df[col].values,normalize=True)))
    
# print(style.BOLD +'Mean acc score : {}'.format(np.mean(acc_score_2)) +style.END) 

Mean acc score : 0.9928782535968987


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
# evaluate all steps on test set
predicted = pipeline.predict(X_test)

In [13]:
#sample 
output=pipeline.predict(msg)
print(y_train.columns.values[(output.flatten()==1)],output)

['related' 'weather_related'] [[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]]


In [14]:
predicted_df=pd.DataFrame(predicted,columns=df.columns[4:])

In [15]:
for col in df.columns[4:]:
    print(style.BOLD + 'Feature : {}\n'.format(col) + style.END,classification_report(y_test[col],predicted_df[col]), '\n')
     

Feature : related
              precision    recall  f1-score   support

          0       0.63      0.47      0.54      1550
          1       0.84      0.92      0.88      4951
          2       0.75      0.17      0.28        53

avg / total       0.79      0.80      0.79      6554
 

Feature : request
              precision    recall  f1-score   support

          0       0.89      0.98      0.93      5415
          1       0.79      0.42      0.55      1139

avg / total       0.87      0.88      0.86      6554
 

Feature : offer
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554
 

Feature : aid_related
              precision    recall  f1-score   support

          0       0.74      0.86      0.80      3815
          1       0.76      0.59      0.66      2739

avg / total       0.75      0.75      0.74      6554
 

F

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Mean Fbeta and accuracy score**

In [39]:
print(style.BOLD + 'F-beta score for the features in Test data\n' + style.END)
f_score_test=[]
for col in df.columns[4:]:
    f_score_test.append(fbeta_score(y_test[col],predicted_df[col],average='weighted',beta=1))
    print(style.BOLD + col + style.END,':', fbeta_score(y_test[col],predicted_df[col],average='weighted',beta=1))    

print(style.BOLD +'\nMean fbeta score : {}'.format(np.mean(f_score_test)) +style.END)        

F-beta score for the features in Test data

related : 0.791148836823
request : 0.863364339568
offer : 0.994053377636
aid_related : 0.741879922313
medical_help : 0.887535970121
medical_products : 0.931465586518
search_and_rescue : 0.964831441116
security : 0.973982521707
military : 0.960553880934
child_alone : 1.0
water : 0.93906388137
food : 0.911196557776
shelter : 0.918580430265
clothing : 0.985168406783
money : 0.968966966055
missing_people : 0.982307436534
refugees : 0.953148722607
death : 0.946028957435
other_aid : 0.808883515653
infrastructure_related : 0.904530387425
transport : 0.938528055654
buildings : 0.929591275989
electricity : 0.971567832604
tools : 0.991311449651
hospitals : 0.984767603168
shops : 0.993596272417
aid_centers : 0.983703587249
other_infrastructure : 0.93397284087
weather_related : 0.855433864656
floods : 0.931388559772
storm : 0.92856390038
fire : 0.984675473984
earthquake : 0.963951127328
cold : 0.978908793844
other_weather : 0.92657080108
direct_report : 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [55]:
acc_score_test=[]
for col in df.columns[4:]:
    acc_score_test.append((predicted_df[col].values==y_test[col].values).mean())
    
print(style.BOLD +'Mean acc score : {}'.format(np.mean(acc_score_test)) +style.END)  

Mean acc score : 0.9438977045400604


In [56]:
print(style.BOLD +'Model1, Mean acc score : {} and Mean fbeta score : {}'.format(np.mean(acc_score_test),np.mean(f_score_test)) +style.END) 

Model1, Mean acc score : 0.9438977045400604 and Mean fbeta score : 0.9315437667156554


### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [23]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f402b828b70>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [24]:
# parameters = {'features__text_pipeline__vect__min_df': [1, 5],
#              'clf__estimator__n_estimators':[10], 
#              'clf__estimator__min_samples_split':[2, 5, 10]}
parameters = {
        'clf__estimator__min_samples_split': [2,4],
        #'clf__estimator__min_samples_leaf': (1,2),
        #'clf__estimator__n_estimators': [10,13],
        'clf__estimator__min_samples_leaf': [2,3],
        'vect__ngram_range': ((1, 1),(1, 2))
    }
cv = GridSearchCV(pipeline, param_grid=parameters,verbose = 10)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), score=0.22612145254806226, total= 2.0min
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), score=0.2128471162648764, total= 2.0min
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  6.3min remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), score=0.2230698809887092, total= 2.0min
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) 


...

[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1), score=0.2230698809887092, total= 2.0min
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 29.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4, vect__ngram_range=(1, 2), score=0.2268843454379005, total= 2.4min
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4, vect__ngram_range=(1, 2), score=0.21071101617332927, total= 2.4min
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=4, vect__ngram_range=(1, 2), score=0.22032346658529142, total= 2.4min
[CV] clf__estimator__min_samples_leaf=3, clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__min_samples_leaf=3, clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), score=0.22276472383277388, total= 1.9min
[CV] clf__estimator__min_samples_leaf=3, clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) 
[

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 79.2min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...1,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__min_samples_split': [2, 4], 'clf__estimator__min_samples_leaf': [2, 3], 'vect__ngram_range': ((1, 1), (1, 2))},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [25]:
cv.cv_results_ 

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

{'mean_fit_time': array([  83.08681798,  108.57746387,   82.75087929,  108.71595677,
          80.49023588,   98.86172462,   80.74821575,   97.68112167]),
 'std_fit_time': array([ 0.64073483,  0.63798242,  0.20129363,  1.43328368,  0.42449748,
         1.72530156,  0.7386794 ,  2.80514721]),
 'mean_score_time': array([ 35.07820559,  35.53067183,  34.64028351,  35.57294122,
         34.33403126,  35.2712543 ,  34.70490456,  35.25027847]),
 'std_score_time': array([ 0.20951894,  0.27554913,  0.25256656,  0.29880888,  0.18222643,
         0.31104054,  0.22764254,  0.12118037]),
 'param_clf__estimator__min_samples_leaf': masked_array(data = [2 2 2 2 3 3 3 3],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_clf__estimator__min_samples_split': masked_array(data = [2 2 4 4 2 2 4 4],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_vect__ngram_range': masked_array(data = [(1, 1) (1,

In [26]:
cv.best_params_

{'clf__estimator__min_samples_leaf': 2,
 'clf__estimator__min_samples_split': 2,
 'vect__ngram_range': (1, 1)}

In [27]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...1,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=1))])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
tuned_pred = cv.predict(X_test)

In [29]:
tuned_df=pd.DataFrame(tuned_pred,columns=df.columns[4:])
tuned_df.head(1)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [43]:
for col in df.columns[4:]:
    print(style.BOLD + 'Feature : {}\n'.format(col) + style.END,classification_report(y_test[col],tuned_df[col]), '\n')

Feature : related
              precision    recall  f1-score   support

          0       0.71      0.40      0.51      1550
          1       0.83      0.95      0.88      4951
          2       0.86      0.11      0.20        53

avg / total       0.80      0.81      0.79      6554
 

Feature : request
              precision    recall  f1-score   support

          0       0.89      0.98      0.93      5415
          1       0.83      0.40      0.54      1139

avg / total       0.88      0.88      0.86      6554
 

Feature : offer
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554
 

Feature : aid_related
              precision    recall  f1-score   support

          0       0.79      0.81      0.80      3815
          1       0.73      0.69      0.71      2739

avg / total       0.76      0.76      0.76      6554
 

F

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [57]:
print(style.BOLD + 'Tuned model 1 :: F-beta score for the features in Test data\n' + style.END)
f_score_test_m2=[]
for col in df.columns[4:]:
    f_score_test_m2.append(fbeta_score(y_test[col],tuned_df[col],average='weighted',beta=1))
    print(style.BOLD + col + style.END,':', fbeta_score(y_test[col],tuned_df[col],average='weighted',beta=1))

print(style.BOLD +'\nMean fbeta score : {}'.format(np.mean(f_score_test_m2)) +style.END)      

Tuned model 1 :: F-beta score for the features in Test data

related : 0.790517834927
request : 0.863140072625
offer : 0.994053377636
aid_related : 0.760304698937
medical_help : 0.88246717817
medical_products : 0.923846157997
search_and_rescue : 0.96224307834
security : 0.974440434854
military : 0.958702309726
child_alone : 1.0
water : 0.912099213949
food : 0.890878033797
shelter : 0.904270894626
clothing : 0.978065918854
money : 0.968224250597
missing_people : 0.981728047728
refugees : 0.953877887558
death : 0.942765488997
other_aid : 0.805672619555
infrastructure_related : 0.904703417557
transport : 0.936285593412
buildings : 0.926108642264
electricity : 0.970240330672
tools : 0.991311449651
hospitals : 0.98492020124
shops : 0.993596272417
aid_centers : 0.983779880128
other_infrastructure : 0.934277985742
weather_related : 0.865505862663
floods : 0.925682367941
storm : 0.925333869879
fire : 0.983613148893
earthquake : 0.956146517389
cold : 0.976699024501
other_weather : 0.92643392082

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [60]:
acc_score_test_m2=[]
for col in df.columns[4:]:
    acc_score_test_m2.append((tuned_df[col].values==y_test[col].values).mean())
    
print(style.BOLD +'Mean acc score : {}'.format(np.mean(acc_score_test_m2)) +style.END)  

Mean acc score : 0.9434272539246602


In [59]:
print(style.BOLD +'Model1, Mean acc score : {} and Mean fbeta score : {}'.format(np.mean(acc_score_test),np.mean(f_score_test)) +style.END) 

Model1, Mean acc score : 0.9438977045400604 and Mean fbeta score : 0.9315437667156554


In [58]:
print(style.BOLD +'Model2(using gridsearch), Mean acc score : {} and Mean fbeta score : {}'.format(np.mean(acc_score_test_m2),np.mean(f_score_test_m2)) +style.END) 

Model2(using gridsearch), Mean acc score : 0.9434272539246602 and Mean fbeta score : 0.9287014092410997


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

We shall use AdaBoostClassifier instead of RandomBoostClassifier.

In [61]:
pipeline_2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=1))),
])

In [7]:
pipeline_2.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__algorithm', 'clf__estimator__base_estimator', 'clf__estimator__learning_rate', 'clf__estimator__n_estimators', 'clf__estimator__random_state', 'clf__estimator', 'clf__n_jobs'])

In [8]:
pipeline_2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f0448159620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=1),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(

In [9]:
parameters_2 = {
        'clf__estimator__n_estimators': [50,100],
        'clf__estimator__learning_rate': [0.07,0.05]
    }
cv_2 = GridSearchCV(pipeline_2, param_grid=parameters_2,verbose = 10)
cv_2.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__learning_rate=0.07, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.07, clf__estimator__n_estimators=50, score=0.21361000915471468, total= 2.6min
[CV] clf__estimator__learning_rate=0.07, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.7min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.07, clf__estimator__n_estimators=50, score=0.21849252364967958, total= 2.6min
[CV] clf__estimator__learning_rate=0.07, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.5min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.07, clf__estimator__n_estimators=50, score=0.20735428745804088, total= 2.6min
[CV] clf__estimator__learning_rate=0.07, clf__estimator__n_estimators=100 


...

[CV]  clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=50, score=0.2004882514494965, total= 2.6min
[CV] clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 36.9min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=100, score=0.21315227342081172, total= 3.6min
[CV] clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=100, score=0.21925541653951786, total= 3.6min
[CV] clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=100, score=0.2079646017699115, total= 3.6min


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 51.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...stimator=None,
          learning_rate=1.0, n_estimators=50, random_state=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [50, 100], 'clf__estimator__learning_rate': [0.07, 0.05]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [10]:
cv_2.best_params_

{'clf__estimator__learning_rate': 0.07, 'clf__estimator__n_estimators': 100}

In [11]:
cv_2.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...imator=None,
          learning_rate=0.07, n_estimators=100, random_state=1),
           n_jobs=1))])

In [13]:
tuned_pred_2 =cv_2.predict(X_test)

In [14]:
tuned_pred_2_df=pd.DataFrame(tuned_pred_2,columns=df.columns[4:])

In [19]:
f_score_test_2=[]
for col in df.columns[4:]:
    f_score_test_2.append(fbeta_score(y_test[col],tuned_pred_2_df[col],average='weighted',beta=1))
    print(style.BOLD + col + style.END,':', fbeta_score(y_test[col],tuned_pred_2_df[col],average='weighted',beta=1))    

print(style.BOLD +'\nMean fbeta score : {}'.format(np.mean(f_score_test_2)) +style.END) 


related : 0.650163804807
request : 0.844605815678
offer : 0.994053377636
aid_related : 0.682959408139
medical_help : 0.886150385361
medical_products : 0.929954637329
search_and_rescue : 0.966302086451
security : 0.974440434854
military : 0.957808547145
child_alone : 1.0
water : 0.958445771281
food : 0.937376767161
shelter : 0.929986191651
clothing : 0.987500626063
money : 0.970728171421
missing_people : 0.981651755459
refugees : 0.954648875322
death : 0.942023656544
other_aid : 0.805351588108
infrastructure_related : 0.905384837949
transport : 0.94062281364
buildings : 0.940324000157
electricity : 0.973085081957
tools : 0.991311449651
hospitals : 0.985845213912
shops : 0.993596272417
aid_centers : 0.983779880128
other_infrastructure : 0.934049144947
weather_related : 0.815762251116
floods : 0.948625657738
storm : 0.899099915182
fire : 0.983974390651
earthquake : 0.967402336988
cold : 0.978429185886
other_weather : 0.925906617124
direct_report : 0.781898890923

Mean fbeta score : 0.9250

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
acc_score_test_2=[]
for col in df.columns[4:]:
    acc_score_test_2.append((tuned_pred_2_df[col].values==y_test[col].values).mean())
    
print(style.BOLD +'Mean acc score : {}'.format(np.mean(acc_score_test_2)) +style.END)  

Mean acc score : 0.9415751195198863


The initial untuned model seems to do better.   
Model1, Mean acc score : 0.9438977045400604 and Mean fbeta score : 0.9315437667156554
the inital model is better than the tuned model and the model with Adaboost Classifier. We should note that no additional preprocesing was done which would have yielded a better accuracy score.

### 9. Export your model as a pickle file

In [62]:
pickle.dump(pipeline, open('classifier.pkl', 'wb'))

#loaded_model = pickle.load(open('model.pkl', 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [63]:
loaded_model = pickle.load(open('classifier.pkl', 'rb'))
type(loaded_model)

sklearn.pipeline.Pipeline

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.